## Laboratorio 9: dataset mushrooms

<img src="https://minas.medellin.unal.edu.co/images/Escudo_color.png" width="300">

**Elaborado por:** Catalina Restrepo Salgado

Fundamentos de Analítica

Semestre 2024-1S

Por medio de la presente actividad, se espera entrenar un modelo de clasificación que permita identificar hongos comestibles y venenosos en un dataset que contiene diferentes instancias de hongos de las familias Agaricus y Lepiota. Para realizar dicha tarea se proporcionaron una serie de características, que serán visualizadas más adelante.

En primer lugar, es preciso importar las librerías necesarias para realizar este ejercicio.

In [1]:
# Importación de librerías necesarias
import pandas as pd
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

A continuación, se leen los datos de entrenamiento y prueba proporcionados en dos archivos separados.

In [2]:
# Lectura de los datos de entrenamiento y prueba
train = pd.read_csv('train_dataset.csv')
test = pd.read_csv('test_dataset.csv')

# Separación de las variables dependientes e independientes
X_train = train.drop(columns=['type'], axis=1)
y_train = train['type']

X_test = test.drop(columns=['type'], axis=1)
y_test = test['type']

Observando el dataset de entrenamiento, se identifican un total de 22 variables con las que se espera entrenar el modelo de clasificación, adicionales a la variable `type`, que es la columna donde se encuentran las dos posibles clases en que se pueden identificar los hongos.

In [3]:
# Visualización del dataset de entrenamiento
X_train.head()

,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,x,y,n,t,p,f,c,n,p,e,...,s,w,w,p,w,o,p,k,s,u
1,f,y,g,t,n,f,c,b,p,t,...,s,g,g,p,w,o,p,k,y,d
2,x,y,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,m
3,x,s,w,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,k,a,g
4,x,f,n,t,n,f,c,b,p,t,...,s,p,w,p,w,o,p,n,v,d


In [4]:
# Visualización de las clases del dataset de entrenamiento
y_train.value_counts()

type
e    3167
p    2926
Name: count, dtype: int64

El conjunto de datos tiene un total de 6093 registros, correspondientes a un grupo de hongos de las familias Agaricus y Lepiota, los cuales fueron previamente identificados como comestibles o venenosos (considerando que, para este caso, los hongos de comestibilidad desconocida deben ser clasificados como venenosos).

In [5]:
# Dimensiones del dataset de entrenamiento
train.shape

(6093, 23)

Como se puede observar a continuación, todas las características presentes en el dataset son categóricas, por lo que deben ser convertidas a numéricas con algún tipo de codificación antes de entrenar el modelo.

In [6]:
# Tipo de datos del dataset de entrenamiento
X_train.dtypes

cap_shape                   object
cap_surface                 object
cap_color                   object
bruises                     object
odor                        object
gill_attachment             object
gill_spacing                object
gill_size                   object
gill_color                  object
stalk_shape                 object
stalk_root                  object
stalk_surface_above_ring    object
stalk_surface_below_ring    object
stalk_color_above_ring      object
stalk_color_below_ring      object
veil_type                   object
veil_color                  object
ring_number                 object
ring_type                   object
spore_print_color           object
population                  object
habitat                     object
dtype: object

Para cumplir este propósito será necesario crear un pipeline con dos pasos: transformación de las variables y clasificación. De esta manera se podrá entrenar un modelo sobre el dataset de entrenamiento y predecir sobre el dataset de prueba sin ningún inconveniente.

In [7]:
# Codificación de las variables categóricas
transformer = ColumnTransformer(
    transformers=[
        ('label_encoder', OneHotEncoder(dtype='int'), X_train.columns)
    ]
)

Ahora bien, para el ejercicio de clasificación se empleará un modelo de K-Nearest Neighbors con un parámetro K=3, que se evaluará posteriormente en su resultado.

In [8]:
# Creación del pipeline con el clasificador KNN
pipeline = Pipeline(
    steps=[
        ('transformer', transformer),
        ('classifier', KNeighborsClassifier(n_neighbors=3))
    ]
)

Una vez creado el pipeline, se entrena el modelo con los datos de entrenamiento, como se muestra en la siguiente línea de código.

In [9]:
# Entrenamiento del modelo de clasificación
pipeline.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('label_encoder',
                                                  OneHotEncoder(dtype='int'),
                                                  Index(['cap_shape', 'cap_surface', 'cap_color', 'bruises', 'odor',
       'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color',
       'stalk_shape', 'stalk_root', 'stalk_surface_above_ring',
       'stalk_surface_below_ring', 'stalk_color_above_ring',
       'stalk_color_below_ring', 'veil_type', 'veil_color', 'ring_number',
       'ring_type', 'spore_print_color', 'population', 'habitat'],
      dtype='object'))])),
                ('classifier', KNeighborsClassifier(n_neighbors=3))])

Dados estos resultados, se evaluarán por medio de una métrica de exactitud para conocer su desempeño, tanto en el conjunto de datos de entrenamiento como en el de prueba.

In [10]:
# Métrica para los datos de entrenamiento
y_pred = pipeline.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)

print(f'Accuracy: {accuracy}')

Accuracy: 1.0


Luego, se realiza la clasificación para el dataset de prueba, a fin de observar si se obtuvieron resultados favorables.

In [11]:
y_pred = pipeline.predict(X_test)

In [12]:
# Métrica para los datos de prueba
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')

Accuracy: 1.0


Después de evaluar el modelo, se considera conveniente guardarlo en disco para su uso posterior. Los resultados obtenidos deben ser evaluados con otro tipo de métricas a fin de comprender por qué la exactitud dio un resultado de 1.0, puesto que es una respuesta que no brinda información suficiente para saber si el modelo es realmente bueno.

In [13]:
# Guardar el modelo
with open('model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)